<a href="https://colab.research.google.com/github/JamesAllen-2000/Sentiment-Analysis/blob/main/Sentiment_Analysis_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install tensorflow-text


In [2]:
pip install tensorflow-hub

In [4]:
pip install keras

In [5]:
pip uninstall tensorflow tensorflow-text


Found existing installation: tensorflow 2.15.1
Uninstalling tensorflow-2.15.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.15.1
Found existing installation: tensorflow-text 2.16.1
Uninstalling tensorflow-text-2.16.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tensorflow_text-2.16.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow_text/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-text-2.16.1


In [1]:
pip install tensorflow==2.12 tensorflow-text==2.12


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow_datasets as tfds

In [3]:
def load_imdb_data():
    (train_data, test_data), info = tfds.load(
        'imdb_reviews',
        split=(tfds.Split.TRAIN, tfds.Split.TEST),
        as_supervised=True,
        with_info=True
    )
    return train_data, test_data, info

In [4]:
def preprocess_text(text):
    return tf.constant([text])

In [5]:
def build_model():
    text_input = Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
        name='preprocessing'
    )
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
        trainable=True,
        name='BERT_encoder'
    )
    outputs = encoder(encoder_inputs)
    pooled_output = outputs['pooled_output']
    dropout = Dropout(0.1)(pooled_output)
    output = Dense(1, activation='sigmoid', name='classifier')(dropout)

    model = Model(inputs=[text_input], outputs=[output])
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def train_model(model, train_data, test_data):
    train_data = train_data.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
    test_data = test_data.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

    history = model.fit(
        train_data,
        validation_data=test_data,
        epochs=3
    )
    return model, history

In [7]:
def analyze_sentiment(model, text):
    inputs = preprocess_text(text)
    prediction = model.predict(inputs)
    return "POSITIVE" if prediction[0][0] > 0.5 else "NEGATIVE", prediction[0][0]

In [ ]:
if __name__ == "__main__":
    train_data, test_data, info = load_imdb_data()
    model = build_model()
    print(model.summary())
    model, history = train_model(model, train_data, test_data)
    text = input("Enter the text you want to analyze: ")
    label, score = analyze_sentiment(model, text)
    print(f"Label: {label}, Score: {score}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePEB5YK/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePEB5YK/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePEB5YK/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids'